In [37]:
import pandas as pd
import datetime
import numpy as np

In [17]:
file_path = '/Users/dmitrijdolgopolov/Documents/1_study_python/projects/1_avito_parsing/flats_from_avito'

In [95]:
current_date = datetime.datetime.today().strftime('%d.%m.%Y')
file_name = f'flats_from_avito_{current_date}.csv'

In [96]:
df = pd.read_csv(file_path + '/' + file_name)

In [58]:
def fix_floor(x):
    'Исправляет опечатку в этажности'
    if x == 99:
        x = 9
        return x
    else:
        return x

In [100]:
df = df.drop(columns='Unnamed: 0')

In [106]:
df.dtypes

url                 object
Cost                 int64
Rooms                int64
Square             float64
Cost_per_m2          int64
Floor                int64
Floors_in_house      int64
Address             object
Metro               object
Metro_distance      object
Date                object
Date_note           object
dtype: object

In [102]:
# исправляет опечатку в этажности
df.Floors_in_house = df.Floors_in_house.apply(fix_floor)

In [109]:
# средние, мин и макс цены за 1 кв.м в домах разной этажности
costs_by_floors_in_house = df.groupby('Floors_in_house', as_index=False). \
                              agg({'Cost_per_m2': ['mean', 'min', 'max']}). \
                              sort_values(('Cost_per_m2', 'mean'))

**Девятиэтажные дома**

In [110]:
# фильтрую по 9-этажным домам
df_9_floors_houses = df.query('Floors_in_house == 9')

In [111]:
# число объявлений
df_9_floors_houses.url.nunique()

147

In [112]:
# проверяю повторы
df_9_floors_houses.duplicated().sum()

0

In [74]:
7600000 / 44.1

172335.60090702947

In [91]:
173000 * 44.1

7629300.0

In [114]:
# нахожу квантили
quantiles = df_9_floors_houses.Cost_per_m2.quantile([0.25, 0.5, 0.75, 0.815])
quantiles

0.250    145928.50
0.500    155577.00
0.750    166312.00
0.815    172889.64
Name: Cost_per_m2, dtype: float64

In [115]:
df_9_floors_houses[df_9_floors_houses.Cost_per_m2 > quantiles[0.750]] \
    .sort_values('Cost_per_m2', ascending=False) \
    .to_csv('''/Users/dmitrijdolgopolov/Documents/1_study_python/projects/1_avito_parsing/flats_from_avito/top/9_floors_houses_0.75_percentile.csv''')

In [68]:
df_9_floors_houses.head()

,url,Cost,Rooms,Square,Floor,Floors_in_house,Address,Metro,Date,Date_note,cost_per_m2
1,https://www.avito.ru/sankt-peterburg/kvartiry/...,7279000,2,50.6,8,9,"ул. Дыбенко, 12к3",Улица Дыбенко,13.04.2023,39 минут назад,143853
6,https://www.avito.ru/sankt-peterburg/kvartiry/...,4562790,2,50.2,6,9,"Товарищеский пр-т, 2к2",Проспект Большевиков,13.04.2023,35 минут назад,90892
9,https://www.avito.ru/sankt-peterburg/kvartiry/...,6800000,2,50.9,5,9,"ул. Дыбенко, 12к1",Улица Дыбенко,13.04.2023,12 часов назад,133595
15,https://www.avito.ru/sankt-peterburg/kvartiry/...,8000000,2,50.2,5,9,"Санкт-Петербург, Товарищеский проспект, 2к2",Проспект Большевиков,12.04.2023,1 день назад,159362
31,https://www.avito.ru/sankt-peterburg/kvartiry/...,7350000,2,51.0,3,9,"ул. Подвойского, 17к2",Улица Дыбенко,07.04.2023,6 дней назад,144117


7688279.376712329